In [1]:
import numpy as np
import os
import shutil
import cv2

# Load Dataset and preprocess each image before model training:

In [2]:
# RESIZE each image to a image of 100*100
def readImage(imageName):
    img =cv2.imread(imageName)
    img = cv2.resize(img,(100,100))
    cv2.imwrite(imageName,img)
for j in range (65,91):
    for i in range (1,3001):
        readImage("dataset/"+chr(j)+"/" + chr(j) +str(i) + '.jpg')


In [3]:
# Split the dataset into training and test set.
# The dataset for each gesture is split into training and test set where test set consists of 25% of the dataset
rootdir= 'D:\Hand Gesture Recognition\dataset'
os.makedirs(rootdir+'/Training_Set')
os.makedirs(rootdir+'/Test_Set')
for i in range(65,91):
    os.makedirs(rootdir +'/Training_Set/' + chr(i))
    os.makedirs(rootdir +'/Test_Set/' + chr(i))
    source = rootdir + '/' + chr(i)
    allFileNames = os.listdir(source)
    np.random.shuffle(allFileNames)
    test_ratio = 0.25 #size of test set
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)* (1 - test_ratio))])
    train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]
    for name in train_FileNames:
      shutil.copy(name, rootdir +'/Training_Set/' + chr(i))
    for name in test_FileNames:
      shutil.copy(name, rootdir +'/Test_Set/' +chr(i))

In [4]:
# Preprocess each image to a greyscale image and load it for model training
#The below function reads the images within each folder of training set and test set.
#It also greyscales each image before storing it for better model training
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
def load_images_from_folder(folder,ch):
    images = []
    for filename in os.listdir(folder+ch):
        img = cv2.imread(os.path.join(folder+ch,filename))
        if img is not None:
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            class_num=labels.index(ch)
            #image_norm = cv2.normalize(gray_image, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
            images.append([gray_image,class_num])
            #cv2.imwrite(folder+'GrayImages/'+ch+'/'+filename,gray_image)
    return images
trainImages = []
#os.makedirs('C:/Users/OMEN/Desktop/dataset/Training_Set/GrayImages')
#os.makedirs('C:/Users/OMEN/Desktop/dataset/Test_Set/GrayImages')
for i in range(65,91):
    #os.makedirs('C:/Users/OMEN/Desktop/dataset/Training_Set/GrayImages/'+chr(i))
    trainImages=trainImages+(load_images_from_folder('D:\Hand Gesture Recognition\dataset',chr(i)))
testImages=[]
for i in range(65,91):
    #os.makedirs('C:/Users/OMEN/Desktop/dataset/Test_Set/GrayImages/'+chr(i))
    testImages=testImages+(load_images_from_folder('D:\Hand Gesture Recognition\dataset',chr(i)))


In [5]:
x_train = []
y_train = []
x_test = []
y_test = []

for feature,label in trainImages:
  x_train.append(feature)
  y_train.append(label)

for feature, label in testImages:
  x_test.append(feature)
  y_test.append(label)

In [6]:
x_test=np.array(x_test)
x_train=np.array(x_train)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

# Nomalizing the image for faster training of our ANN model:


In [7]:
x_train=x_train/255.0
x_test=x_test/255.0
y_train = np.array(y_train)
y_train=y_train.reshape(-1,1)
y_test = np.array(y_test)
y_test=y_test.reshape(-1,1)


In [16]:
y_tt=y_test.flatten()
x_tt=[]
for i in range(0,19500):
    x_tt.append(x_test[i].flatten())
x_tt=np.array(x_tt)


In [8]:
y_tr=y_train.flatten()
x_tr=[]
for i in range(0,58500):
    x_tr.append(x_train[i].flatten())
x_tr=np.array(x_tr)

In [15]:

print(np.shape(y_test))

(19500, 1)


# Implementing the ANN model:

In [11]:
#Building and Training our ANN model
import tensorflow as tf
ANN=tf.keras.models.Sequential()
ANN.add(tf.keras.layers.Dense(units=300,activation='relu'))
ANN.add(tf.keras.layers.Dense(units=100,activation='relu'))
ANN.add(tf.keras.layers.Dense(units=26,activation='softmax'))
ANN.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [12]:
ANN.fit(x_tr,y_tr,batch_size=32,epochs=100)

Epoch 1/100
1829/1829 [==============================] - 38s 20ms/step - loss: 2.5621 - accuracy: 0.2104
Epoch 2/100
1829/1829 [==============================] - 43s 23ms/step - loss: 1.5817 - accuracy: 0.4617
Epoch 3/100
1829/1829 [==============================] - 37s 20ms/step - loss: 1.2186 - accuracy: 0.5742
Epoch 4/100
1829/1829 [==============================] - 43s 24ms/step - loss: 1.0252 - accuracy: 0.6338
Epoch 5/100
1829/1829 [==============================] - 36s 20ms/step - loss: 0.9074 - accuracy: 0.6729
Epoch 6/100
1829/1829 [==============================] - 35s 19ms/step - loss: 0.8115 - accuracy: 0.7044
Epoch 7/100
1829/1829 [==============================] - 34s 19ms/step - loss: 0.7565 - accuracy: 0.7246
Epoch 8/100
1829/1829 [==============================] - 35s 19ms/step - loss: 0.7108 - accuracy: 0.7407
Epoch 9/100
1829/1829 [==============================] - 34s 19ms/step - loss: 0.6568 - accuracy: 0.7596
Epoch 10/100
1829/1829 [==============================]

In [13]:
import joblib
ANN.save("ANN_MODEL.h5")

In [18]:
import joblib
ANN_model=tf.keras.models.load_model("ANN_MODEL.h5")
y_pred = ANN.predict(x_tt)
c=np.argmax(y_pred, axis=1)

610/610 [==============================] - 3s 5ms/step


In [19]:
np.save("labels.npy", np.array(labels))

# Predicting the test set:

In [16]:
y_pred = ANN.predict(x_test)
c=np.argmax(y_pred, axis=1)

610/610 [==============================] - 1s 901us/step


In [20]:
print(np.concatenate((c.reshape(len(c),1), y_tt.reshape(len(y_tt),1)),1))

[[ 0  0]
 [ 0  0]
 [ 0  0]
 ...
 [25 25]
 [25 25]
 [25 25]]


In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_tt,c)
print(cm)

[[665   5   1   7  27   9   0   0   0   0   0   1   9   0  20   0   0   0
    2   2   0   0   0   0   0   2]
 [  8 719   0   1  11   1   0   0   0   0   0   0   1   0   1   0   0   0
    0   0   4   0   4   0   0   0]
 [  5   0 712  27   2   1   0   0   0   0   0   0   1   0   2   0   0   0
    0   0   0   0   0   0   0   0]
 [  6   6   8 702   4   6   0   0   0   0   0   0   0   0  18   0   0   0
    0   0   0   0   0   0   0   0]
 [ 10  21   0   5 696   3   0   0   0   0   0   0   0   0   5   0   0   0
    2   0   5   0   3   0   0   0]
 [  5   3   0   7  22 701   5   0   0   0   0   0   0   0   1   0   0   0
    0   1   1   0   1   1   2   0]
 [  0   0   0   0   0   0 742   3   0   3   0   0   0   0   0   0   0   0
    0   0   1   0   0   0   1   0]
 [  0   0   0   0   0   0  23 713   2   1   0   0   0   0   0   8   3   0
    0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 673   3  38   0   1   0   0   1   0  14
    0   0   2   6   4   8   0   0]
 [  0   0   0   0  